In [ ]:
from IPython.display import display
import pandas as pd
import csv
import random
import re
# import pyperclip
import numpy as np
import seaborn as sns
import requests
import matplotlib, matplotlib.pyplot as plt
import pprint
import os
import dateutil
import datetime

In [ ]:
pd.options.display.max_rows = 10
sns.set()
matplotlib.rcParams['font.sans-serif'] = ['Arial Unicode MS', 'sans-serif']

In [ ]:
def to_number(cell):
    if not isinstance(cell, str):
        return cell
    _ = re.findall(r'\d+', cell)
    if not _:
        return cell
    return _[0]

In [ ]:
def to_date(cell):
    if not isinstance(cell, str):
        return cell
    return '-'.join(re.findall(r'\d+', cell))

In [ ]:
df = pd.read_excel('赛题1数据材料/赛题1数据集/工商基本信息表.xlsx', index_col='企业编号')
df['经营状态'][df['经营状态'] != '迁出'] = '存续（在营、开业、在册）'
df['行业小类（代码）'] = df['行业小类（代码）'].map(to_number)
df['城市代码'] = df['城市代码'].map(lambda x: x % 100)
df['经营期限自'] = df['经营期限自'].map(to_date)
df['发照日期'] = df['发照日期'].map(to_date)
df.columns=['$注册资本（万元）', '注册资本币种(正则)', '成立日期', '经营状态', '行业大类（代码）', '行业小类（代码）', '类型',
       '省份代码', '城市代码', '地区代码', '是否上市', '经营期限自', '经营期限至', '登记机关区域代码', '发照日期',
       '员工人数', '注销原因', '注销时间']
df

## 筛选数据

In [ ]:
registered_capital_ten_thousand_yuan = df[['$注册资本（万元）']].groupby('企业编号').mean().T
registered_capital_ten_thousand_yuan = registered_capital_ten_thousand_yuan.reindex(columns=list(range(1001, 4001)))
registered_capital_ten_thousand_yuan

In [ ]:
registered_capital_currency_regular = pd.get_dummies(df['注册资本币种(正则)'].replace('-', None), prefix='注册资本币种(正则)').groupby('企业编号').mean().T
registered_capital_currency_regular = registered_capital_currency_regular.reindex(columns=list(range(1001, 4001))).fillna(0)
registered_capital_currency_regular

In [ ]:
def to_day(cell):
    if not isinstance(cell, str):
        return cell
    return (dateutil.parser.parse(cell) - datetime.datetime(2019, 3, 5)).days

In [ ]:
date_of_establishment = df[['成立日期']].applymap(to_day).groupby('企业编号').mean().T
date_of_establishment = date_of_establishment.reindex(columns=list(range(1001, 4001)))
date_of_establishment

In [ ]:
business_status = pd.get_dummies(df['经营状态'], prefix='经营状态').groupby('企业编号').mean().T
business_status = business_status.reindex(columns=list(range(1001, 4001))).fillna(0)
business_status

In [ ]:
industry_category_code = pd.get_dummies(df['行业大类（代码）'], prefix='行业大类（代码）').groupby('企业编号').mean().T
industry_category_code = industry_category_code.reindex(columns=list(range(1001, 4001))).fillna(0)
industry_category_code

In [ ]:
industry_subclass_code = pd.get_dummies(df['行业小类（代码）'], prefix='行业小类（代码）').groupby('企业编号').mean().T
industry_subclass_code = industry_subclass_code.reindex(columns=list(range(1001, 4001))).fillna(0)
industry_subclass_code

In [ ]:
types = pd.get_dummies(df['类型'], prefix='类型').groupby('企业编号').mean().T
types = types.reindex(columns=list(range(1001, 4001))).fillna(0)
types

In [ ]:
provincial_code = pd.get_dummies(df['省份代码'], prefix='省份代码').groupby('企业编号').mean().T
provincial_code = provincial_code.reindex(columns=list(range(1001, 4001))).fillna(0)
provincial_code

In [ ]:
city_code = pd.get_dummies(df['城市代码'], prefix='城市代码').groupby('企业编号').mean().T
city_code = city_code.reindex(columns=list(range(1001, 4001))).fillna(0)
city_code

In [ ]:
whether_it_is_listed = pd.get_dummies(df['是否上市'], prefix='是否上市').groupby('企业编号').mean().T
whether_it_is_listed = whether_it_is_listed.reindex(columns=list(range(1001, 4001))).fillna(0)
whether_it_is_listed

In [ ]:
operating_period_from = df[['经营期限自']].applymap(to_day).groupby('企业编号').mean().T
operating_period_from = operating_period_from.reindex(columns=list(range(1001, 4001)))
operating_period_from

In [ ]:
date_of_issue = df[['发照日期']].applymap(to_day).groupby('企业编号').mean().T
date_of_issue = date_of_issue.reindex(columns=list(range(1001, 4001)))
date_of_issue

In [ ]:
number_of_workers = df[['员工人数']].groupby('企业编号').mean().T
number_of_workers = number_of_workers.reindex(columns=list(range(1001, 4001)))
number_of_workers

In [ ]:
reason_for_cancellation = pd.get_dummies(df['注销原因'], prefix='注销原因').groupby('企业编号').mean().T
reason_for_cancellation = reason_for_cancellation.reindex(columns=list(range(1001, 4001))).fillna(0)
reason_for_cancellation

In [ ]:
pd.concat([registered_capital_ten_thousand_yuan, registered_capital_currency_regular, date_of_establishment, 
               business_status, industry_category_code, industry_category_code, types, provincial_code, city_code, 
               whether_it_is_listed, operating_period_from, date_of_issue, number_of_workers, 
               reason_for_cancellation]).to_pickle('工商基本信息表.pickle')